In [50]:
import numpy as np
from tqdm import tqdm
from collections import deque
import timeit
with open("input_12") as f:
    input = np.asarray([np.asarray([i for i in line]) for line in f.read().split("\n")])

In [ ]:
alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
len(alphabet)

In [47]:
def find_areas_1(crop_type, input):
    crops = np.where(input == crop_type)
    lots = list(zip(crops[0].tolist(), crops[1].tolist()))
    areas = []
    while len(lots) > 0:
        current_area = {lots[0]}
        del lots[0]
        prev_len = 0
        while len(current_area) > prev_len:
            prev_len = len(current_area)
            for i, lot in enumerate(lots):
                neighbors = [(lot[0]+1,lot[1]),(lot[0]-1,lot[1]),(lot[0],lot[1]+1),(lot[0],lot[1]-1)]
                for neighbor in neighbors:
                    if neighbor in current_area:
                        current_area.add(lot)
                        del lots[i]
                        break
        areas.append(current_area)
    return areas

In [56]:
def find_areas_2(crop_type, input):
    crops = np.where(input == crop_type)
    lots = list(zip(crops[0].tolist(), crops[1].tolist()))
    areas = []
    queue = deque()
    while len(lots) > 0:
        current_area = {lots[0]}
        queue.append(lots[0])
        del lots[0]
        while len(queue) > 0:
            el = queue.pop()
            neighbors = [(el[0]+i,el[1]) for i in (-1,1)]
            neighbors += [(el[0],el[1]+i) for i in (-1,1)]
            for neighbor in neighbors:
                if neighbor in lots:
                    current_area.add(neighbor)
                    queue.append(neighbor)
                    lots.remove(neighbor)
        areas.append(current_area)
    return areas


In [53]:
#solution with scipy:
from scipy.ndimage import label
def find_areas_3(crop_type, input):
    np.putmask(input, input != crop_type, 0)
    np.putmask(input, input == crop_type, 1)
    mask = np.astype(input, int)
    labels, _ = label(mask)
    areas = [np.where(labels == i) for i in range(1, labels.max() + 1)]
    return [list(zip(area[0], area[1])) for area in areas]
        

In [23]:
def perimeter(area):
    perimeter = 0
    for coord in area:
        for i in (-1, 1):
            if (coord[0] + i, coord[1]) not in area:
                perimeter += 1
            if (coord[0], coord[1] + i) not in area:
                perimeter += 1
    return perimeter


In [ ]:
%timeit for letter in alphabet: find_areas(letter, input.copy())

In [ ]:
%timeit for letter in alphabet: find_areas_2(letter, input.copy())

In [ ]:
%timeit for letter in alphabet: find_areas_3(letter, input.copy())

In [ ]:
c = 0
for letter in tqdm(alphabet):
    areas = find_areas_3(letter, input.copy())
    for area in areas:
        price = len(area) * perimeter(area)
        c += price
c

In [12]:
def perimeter_2(area, N):
    def upper_perimeter(area, N):
        perimeter = 0
        for i in range(N):
            last = False
            for j in range(N):
                if (i,j) in area and (i-1,j) not in area:
                    if not last:
                        perimeter += 1
                        last = True
                else:
                    last = False
        return perimeter
    perimeter = 0
    def rot(area, N):
        rotated_area = []
        for coord in area:
            new_coord = (N-coord[1]-1, coord[0])
            rotated_area.append(new_coord)
        return rotated_area
    for _ in range(4):
        perimeter += upper_perimeter(area, N)
        area = rot(area, N)
    return perimeter

In [ ]:
c = 0
for letter in tqdm(alphabet):
    areas = find_areas(letter, input.copy())
    for area in areas:
        price = len(area) * perimeter_2(area, len(input))
        c += price
c